# Polling With Two State Machines

One team asked how they should incorporate a sensor into their design. Some sensors have an interface that requires you to check periodically if the sensor value changed in a specific way that is interesting to you. For instance, a passive infrared sensor (PIR) may require that you poll regularly if it detected a motion. Or you want to check if a tmperature sensor has detected a change in temperature. 

When we check  a sensor periodically, we also call that **polling**. In a state machine, this is easy to implement with a transition that is periodocally triggered by a timer. Of course, you can integrate polling into the state machine for your main logic. But this can make the main state machine quite complex. Instead, you can also do the polling in a separate state machine, and only send a signal to another state machine if you detected a significant change. 

This notebook is walking through an example step by step. You can execute each cell that contains Python code cell by cell. Just select the cell, then press Shift + Enter. This executes the cell and moves to the next one.

Let's first get STMPY and the other libraries from the basement. When this tutorial is life, you should have STMPY 0.5 at least. If you havent, update. Otherwise this will not work.

In [ ]:
from stmpy import Machine, Driver
import stmpy
import ipywidgets as widgets
from IPython.display import display
print(stmpy.__version__)

We want to periodically check the temperature. Just for this demonstration, we use a global variable that keeps the temperature. We do this by a global variable. For some reason it only seems to work when we use a list and update the variable by adding elements to that list. THis is more a technical detail. Just imagine that this global variable represents the temperature we want to measure.

(You need to execute all Python cells, so also the next one.)

In [ ]:
global temperature
temperature = [21.0]

Now we need some way for you to change the temperature. For this we use a slider widget. The widgets are documented [here][widget].

[widget]: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [ ]:
def handle_slider_change(slider_value):
    # change the global variable by appending the new value to the list
    temperature.append(slider_value['new'])


# display a horizontal slider
temp_slider = widgets.FloatSlider(value=21.0, min=-10, max=40.0, step=0.5, description='Temp:', continuous_update=True)
temp_slider.observe(handle_slider_change, names='value')
display(temp_slider)

Adjust the temperature above, then execute the cell below. You should see that the global variable of the temperature changes. (Index -1 means we are getting the last value of the list.)

In [ ]:
temperature[-1]

Now we construct a state machine that constantly polls the value of the temperature, and that creates a signal when the temperature changes. We use a compound transition for this, implemented by a Python function. Execute the cell below. It only builds the state machine, but does not yet start it, so nothing will be visible. 

In [ ]:
class Polling:
    
    def __init__(self):
        self.last_temp = -100.0
    
    def check_temperature(self):
        # reading the global variable (taking last element)
        temp = temperature[-1]
        if self.last_temp != temp:
            # we detected a change, so send a signal!
            self.stm.driver.send('temp_change', 'stm_receiver', args=[temp])
            print('sending signal')
        self.last_temp = temp
        return 's_polling'
    
polling = Polling()

t0 = {'source': 'initial',
      'target': 's_polling'}

# compound transition
t1 = {'trigger':'timeout',
      'source':'s_polling',
      'function': polling.check_temperature}

s_polling = {'name': 's_polling',
            'entry': 'start_timer("timeout", "1000")'}

stm_polling = Machine(name='stm_polling', transitions=[t0, t1], states=[s_polling], obj=polling)
polling.stm = stm_polling

We also create a state machine that receives the signal when the temperature is updated. In a real system, this would be a more complex state machine that reacts to a temperature change and for instance controls another device. Execute the cell. It should then show a text field that will show the temperature update value as received when we start everything.

In [ ]:
class Receiver:
    
    def __init__(self):
        self.text = widgets.Text(value='', placeholder='', description='Received:', disabled=False)
        display(self.text)
    
    def update_field(self, temp):
        text = '{}'.format(temp)
        self.text.set_trait(name='value', value=text)
    
receiver = Receiver()

t0 = {'source': 'initial',
      'target': 'receiving'}

t1 = {'trigger': 'temp_change',
      'source' : 'receiving',
      'target' : 'receiving',
      'effect' : 'update_field(*)'}

stm_receiver = Machine(name='stm_receiver', transitions=[t0, t1], obj=receiver)
receiver.stm = stm_receiver

Here is a diagram of the two machines communicating with each other:

![](images/polling/polling.png)

Now let's start eveything. Execute the cell below, then play with the slide from above to change the temperature. You should see that the value above only changes when it receives an update from the timer, that means, once every second. 

In [ ]:
driver = Driver()
driver.add_machine(stm_polling)
driver.add_machine(stm_receiver)
driver.start()

(You may want to stop the cell with the stop button above in the menu.)

## What just happened?

Of course, this was a demonstration of two communicating machines. And we used a lot of cosde just for providing the context in which they communicate. So don't make the mistake and think that this is something we do for creating user interfaces for Python. In that case, the solution would be simpler. 

The example showed how one state machine can constantly poll a value, and then prodiuce a simple signal once it found something that another machine is interested in. In this way, you can reduce the complexity of a lot of machines and a lot of concurrent logic.
